In [1]:
# libabries for data exploration.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# load sales data.
sales = pd.read_csv("../../../Data/future_sales/sales_train.csv")

In [3]:
# purpose of this code is to replace dates with equivalent numeric values.
date_dict = dict()
arr = []
counter = 0

for date in sales["date"]:
    if date not in date_dict:
        date_dict[date] = counter 
        counter += 1
    else:
        pass
    
for date in sales["date"]:
    arr.append(date_dict[date])

In [4]:
sales["date_num"] = arr
sales = sales.sort_values("date_num")

In [6]:
# this helps avoid using the separate test set data.
# test set data only used after a compitent model has being built.
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(sales, test_size=0.2, random_state=44)

In [10]:
# training and test data created.
X_train = train_set.loc[:, ["date_num", "shop_id", "item_id", "item_price"]]
y_train = train_set.loc[:, ["item_cnt_day"]]
X_test = test_set.loc[:, ["date_num", "shop_id", "item_id", "item_price"]]
y_test = test_set.loc[:, ["item_cnt_day"]]

In [20]:
# libaries for building linear models.
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error

In [15]:
# linear regression without polynomial features.
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

LinearRegression()

In [22]:
# measures training error.
sales_predictions_train = lin_reg.predict(X_train)
grid_mse = mean_squared_error(sales_predictions_train, y_train)
grid_rmse = np.sqrt(grid_mse)
grid_rmse

2.7303022682486957

In [25]:
# measures test error.
sales_predictions_test = lin_reg.predict(X_test)
grid_mse = mean_squared_error(sales_predictions_test, y_test)
grid_rmse = np.sqrt(grid_mse)
grid_rmse

2.1105118673573418

In [ ]:
# adds additional features to fit the nonlinear data.
poly_feat = PolynomialFeatures(degree=10, include_bias=False)
X_train_poly = poly_feat.fit_transform(X_train)
X_test_poly = poly_feat.fit_transform(X_test)

In [ ]:
# linear regression with polynomial features.
lin_reg_poly = LinearRegression()
lin_reg_poly.fit(X_train_poly, y_train)

In [ ]:
# measures training error.
sales_predictions_train = lin_reg_poly.predict(X_train_poly)
grid_mse = mean_squared_error(sales_predictions_train, y_train)
grid_rmse = np.sqrt(grid_mse)
grid_rmse

In [ ]:
# measures test error.
sales_predictions_test = lin_reg_poly.predict(X_test_poly)
grid_mse = mean_squared_error(sales_predictions_test, y_test)
grid_rmse = np.sqrt(grid_mse)
grid_rmse